# Video Summarization Steps
### Convert frames to base64
### Get audio content
### Summarize the video only
### summarize the audio only
### summarize both audio and video together

In [ ]:
%pip install --upgrade openai --quiet
%pip install opencv-python --quiet
%pip install moviepy --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
open_ai_api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
from openai import OpenAI
import os

model = "gpt-4o"
client = OpenAI(api_key= open_ai_api_key)

In [ ]:
import cv2
from moviepy.editor import VideoFileClip
import time
import base64

In [ ]:
VIDEO_PATH = "/content/keynote_recap.mp4"

In [ ]:
def process_video(video_path, seconds_per_frame =2 ):
  base64frames = []
  base_video_path,_ = os.path.splitext(video_path)
  print(base_video_path)

  video = cv2.VideoCapture(video_path)
  total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
  fps = video.get(cv2.CAP_PROP_FPS)
  frames_to_skip = int(fps*seconds_per_frame)

  curr_frame = 0

  while curr_frame < total_frames - 1:
    video.set(cv2.CAP_PROP_POS_FRAMES, curr_frame)
    sucess, frame = video.read()

    if not sucess:
      break

    _, buffer = cv2.imencode(".jpg", frame)
    base64frames.append(base64.b64encode(buffer).decode("utf-8"))
    curr_frame = curr_frame + frames_to_skip

  video.release()

  audio_path = f"{base_video_path}.mp3"
  clip = VideoFileClip(video_path)
  clip.audio.write_audiofile(audio_path, bitrate="32k")
  clip.audio.close()
  clip.close()

  return base64frames, audio_path

In [ ]:
base64_frames, audio_path = process_video(VIDEO_PATH)

/content/keynote_recap
MoviePy - Writing audio in /content/keynote_recap.mp3


MoviePy - Done.


In [ ]:
base64_frames = base64_frames[::20]

In [ ]:
respose = client.chat.completions.create(
    model = model,
    messages = [
        {
            "role": "system",
            "content": "You are generating a video summary, Please provide a summary of the video, repond in Markdown"
        },
        {
            "role" : "user",
            "content": [
                "There are the frames from the video",
                *map(lambda x: {"type": "image_url",
                                "image_url": {"url": f'data:image/jpg;base64,{x}', "detail": "low"}

                }, base64_frames)
            ]
        }
    ],
    temperature = 0,
)

In [ ]:
print(respose.choices[0].message.content)

## Video Summary

The video appears to be from an event called "OpenAI DevDay." Here is a summary based on the provided frames:

1. **Title Frame**: The video begins with a title screen displaying "OpenAI DevDay."
2. **Function Calling Slide**: A slide is shown explaining the concept of function calling, illustrating a "before" and "after" scenario where commands are transformed into function calls.
3. **Speaker Presentation**: A speaker, dressed in a green sweater and dark pants, is presenting on stage. The background suggests a formal event setting.
4. **Key Points Slide**: The speaker discusses three main points: Instructions, Expanded Knowledge, and Actions, as shown on a slide behind him.
5. **Continued Presentation**: The speaker continues to elaborate on the topics, engaging with the audience.
6. **Closing Gesture**: The speaker waves, possibly indicating the end of the presentation or a transition to another segment.

The video likely covers advancements or features related to 

In [ ]:
transcription = client.audio.transcriptions.create(
    model="whisper-1",
    file = open(audio_path, "rb")
)

In [ ]:
print("Transcript:", transcription.text)

Transcript: Welcome to our first-ever OpenAI Dev Day. Today, we are launching a new model, GPT-4 Turbo. GPT-4 Turbo supports up to 128,000 tokens of context. We have a new feature called JSON mode, which ensures that the model will respond with valid JSON. You can now call many functions at once, and it'll do better at following instructions in general. You want these models to be able to access better knowledge about the world. So do we. So we're launching retrieval in the platform. You can bring knowledge from outside documents or databases into whatever you're building. GPT-4 Turbo has knowledge about the world up to April of 2023, and we will continue to improve that over time. Dolly 3, GPT-4 Turbo with Vision, and the new Text-to-Speech model are all going into the API today. Today, we're launching a new program called Custom Models. With Custom Models, our researchers will work closely with the company to help them make a great custom model, especially for them and their use case

In [ ]:
response = client.chat.completions.create(
    model = model ,
    messages = [
        {
            "role" : "system",
            "content": " You are genrating a transcript summary, create  a summar of the provided transcription"
        },
        {
            "role" : "user",
            "content": [
                {
                    "type" : "text",
                    "text": f"the audio tracription is : {transcription.text}"
                }
            ]
        }
    ],
    temperature = 0

)

In [ ]:
print(response.choices[0].message.content)

At the first-ever OpenAI Dev Day, several significant announcements were made:

1. **GPT-4 Turbo**: A new model supporting up to 128,000 tokens of context, featuring JSON mode for valid JSON responses, improved instruction following, and better knowledge retrieval from external documents or databases. It is also more cost-effective than GPT-4.

2. **New Features and Models**: 
   - **Dolly 3** and **GPT-4 Turbo with Vision**.
   - **Text-to-Speech model**.
   - **Custom Models** program for tailored solutions with higher rate limits and customizable quotas.

3. **GPTs**: Tailored versions of ChatGPT for specific purposes, programmable through conversation, with options for private or public sharing, and a forthcoming GPT Store.

4. **Assistance API**: Includes persistent threads, built-in retrieval, a code interpreter, and improved function calling.

The event emphasized the potential of these technologies to provide "superpowers on demand" and hinted at even more advanced developments

In [ ]:
response = client.chat.completions.create(
    model = model,
    messages = [
        {
            "role": "system",
            "content": " You are generating a video summary, create a summary of the provided video and its transcript. Reponse in Markdown"

        },
        {
            "role": "user",
            "content": [
                'These are the frame from the video',
                 *map(lambda x: {"type": "image_url",
                                "image_url": {"url": f'data:image/jpg;base64,{x}', "detail": "low"}

                }, base64_frames),

                {
                    "type" : "text",
                    "text": f"the audio tracription is : {transcription.text}"
                }

            ]
        }
    ],
    temperature = 0,
)

In [ ]:
print(response.choices[0].message.content)

## Video Summary

### Title: OpenAI Dev Day

#### Key Highlights:

1. **Introduction:**
   - The event is titled "OpenAI Dev Day."
   - The speaker welcomes the audience to the first-ever OpenAI Dev Day.

2. **Launch of GPT-4 Turbo:**
   - A new model, GPT-4 Turbo, is introduced.
   - GPT-4 Turbo supports up to 128,000 tokens of context.
   - A new feature called JSON mode ensures the model responds with valid JSON.
   - The model can call multiple functions simultaneously and follows instructions better.

3. **Knowledge Retrieval:**
   - A new retrieval feature allows bringing knowledge from external documents or databases into applications.
   - GPT-4 Turbo has knowledge up to April 2023, with ongoing improvements.

4. **New Models and Features:**
   - Dolly 3, GPT-4 Turbo with Vision, and a new Text-to-Speech model are available in the API.
   - A new program called Custom Models is launched, allowing companies to create custom models with the help of OpenAI researchers.
   - Higher

In [ ]:
QUESTION = "Question: Why did Sam Altman have an example about raising windows and turning the radio on?"

In [ ]:
response = client.chat.completions.create(
    model = model,
    messages = [
        {
            "role": "system",
            "content": "Use the video to answer the provied question, Respond in Markdown"
        },
        {
            "role" : "user",
            "content": [
                "There are the frames from the video",
                *map(lambda x: {"type": "image_url",
                                "image_url": {"url": f'data:image/jpg;base64,{x}', "detail": "low"}

                }, base64_frames),
                QUESTION
            ],
        }
    ],
    temperature = 0,
)

In [ ]:
print(response.choices[0].message.content)

Sam Altman used the example about raising windows and turning the radio on to illustrate the concept of function calling. The example demonstrates how a command given in natural language ("raise my windows and turn the radio on") can be broken down into specific functions (raise_windows() and radio_on()) that a system can execute. This highlights the capability of advanced AI to understand and perform complex tasks by interpreting and acting on natural language instructions.


In [ ]:
response = client.chat.completions.create(
    model = model ,
    messages = [
        {
            "role" : "system",
            "content": " Use transcription to answer the provided question"
        },
        {
            "role" : "user",
            "content": [
                {
                    "type" : "text",
                    "text": f"the audio tracription is : {transcription.text}. \n\n {QUESTION}"
                }
            ]
        }
    ],
    temperature = 0

)

In [ ]:
print(response.choices[0].message.content)

The provided transcription does not include any mention of Sam Altman or an example about raising windows and turning the radio on. Therefore, I cannot provide an answer to that question based on the given transcription.


In [ ]:
response = client.chat.completions.create(
    model = model,
    messages = [
        {
            "role": "system",
            "content": "Use the video and transcription to anser the provided question"

        },
        {
            "role": "user",
            "content": [
                'These are the frame from the video',
                 *map(lambda x: {"type": "image_url",
                                "image_url": {"url": f'data:image/jpg;base64,{x}', "detail": "low"}

                }, base64_frames),

                {
                    "type" : "text",
                    "text": f"the audio tracription is : {transcription.text}"
                },

                QUESTION

            ],
        }
    ],
    temperature = 0,
)

In [ ]:
print(response.choices[0].message.content)

Sam Altman used the example about raising windows and turning the radio on to illustrate the improved function calling capabilities of the new model, GPT-4 Turbo. The example demonstrates how the model can now handle multiple function calls more effectively and follow instructions better. This improvement allows for more complex and accurate interactions, such as executing multiple commands in a single request.
